<a href="https://colab.research.google.com/github/maja1001/yolov5/blob/master/Einfuehrung_in_die_Objekterkennung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Logo_of_Hochschule_Kaiserslautern.png/320px-Logo_of_Hochschule_Kaiserslautern.png)

![](https://miro.medium.com/v2/resize:fit:640/format:webp/0*mmUHzD_vxDJn5Rfc.gif)

# **Eine Einfuehrung in die Objekterkennung**

Training und Anwendung von künstlichen neuronalen Netzen

*Ein Workshop von Max*

# **1. Einrichten der Umgebung**

- Repository klonen
- Abhänigkeiten installieren *(mit `pip3.x`)*
- Installation Prüfen: print PyTorch und GPU Version

In [ ]:
# !git clone https://github.com/ultralytics/yolov5  # clone
!git clone https://github.com/maja1001/yolov5.git # clone forked repo

%cd yolov5
%pip install -r requirements.txt
%pip install roboflow

print("\n[Info] Repository erfolgreich geklont und Abhänigkeiten installiert.")

In [ ]:
import torch
# import utils
# from roboflow import Roboflow

# display = utils.notebook_init()  # checks
# from IPython.display import Image, clear_output  # to display images

print( f"[Info] Torch Version: {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'Keine Grafikkarte gefunden: CPU wird verwendet'})" )

In [ ]:
!git pull

# **2. Verifizierung der Codebasis**

- Validiere die Installation des Repositories anhand von zwei Bildern
- Einsatz von `yolov5s`

In [ ]:
# Teste detect
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/bus.jpg --name test/bus
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/zidane.jpg --name test/zidane

!python detect.py --weights yolov5s.pt --img 640 --conf 0.7 --source data/images/bus.jpg --name test/busHighConf
!python detect.py --weights yolov5s.pt --img 640 --conf 0.7 --source data/images/zidane.jpg --name test/zidaneHighConf

# test both pictures
# !python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

In [ ]:
# Entferne bisherige Ergebnisse
!rm -r runs/detect/test

# **3. Datensatz einbinden**
- Unter der Verwendung des Roboflow Packages
- Roboflow Schnittstelle mit entsprechenden Schlüssel nutzen
- Hier gehts zu [Roboflow](https://roboflow.com/?ref=ultralytics)

Hier geht es zu einigen Vorbereiteten [Datensätzen](https://universe.roboflow.com/hsklworkshop-m8g4f)

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="vB3ieGw7uAwm1qyRpILr")
project = rf.workspace("hsklworkshop-m8g4f").project("laptop-o9ebb")
dataset = project.version(2).download("yolov5")

# **4. Training**

### Es gibt mehrere Möglichkeiten ein Training zu starten

1. In Google Colab
2. Alternativ über das [Ultralytics HUB](https://hub.ultralytics.com/home)


#### **1. Google Colab**

- Basis Befehl / Python Script = `train.py`
- Mehrere Argumente sind möglich:

```
# Auszug
--img 640
--batch 16
--epochs 100
--cfg ../datasets/train_journey/config/custom_v5s6.yaml
--data ../datasets/train_journey/data/with_aug_5c_1800/data.yaml
--weights v5s6.pt
--hyp ../datasets/train_journey/config/hyp-low-no-mosaic.yaml
--project ../datasets/train_journey/run_v6_new
--name with_aug_5c_1800
--cache ram
```

- Default Werte sind gesetzt
- Beispielhafter Trainings Befehl: `!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache ram`

In [ ]:
# Mit MS COCO
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache ram

In [ ]:
# Individueller Datensatz
!python train.py --img 640 --batch 64 --epochs 75 --cfg custom/yolov5sCustomOneClass.yaml --data small_smartphone_dataset_merged-4/data.yaml --weights yolov5s.pt --hyp custom/hypScratchMedNoMosaic.yaml --project small_smartphone_dataset_merged-4/result --name run_1 --cache ram

In [ ]:
# Individueller Datensatz
!python train.py --img 640 --batch 64 --epochs 75 --cfg custom/yolov5sCustomThreeClasses.yaml --data bottle_paper_carton_merge-3/data.yaml --weights yolov5s.pt --hyp custom/hypScratchMedNoMosaic.yaml --project bottle_paper_carton_merge-3/result --name run_1 --cache ram

In [ ]:
# Individueller Datensatz
!python train.py --img 640 --batch -1 --epochs 75 --cfg custom/yolov5sCustomOneClass.yaml --data laptop-1/data.yaml --weights yolov5s.pt --hyp custom/hypScratchMedNoMosaic.yaml --project laptop-1/result --name run_2 --cache ram

In [ ]:
!python train.py -h

In [ ]:
# Entferne Trainings Ergebnisse
!rm -r small_smartphone_dataset-1/result

#### **2. Ultralytics HUB**

- HUB mit Roboflow linken
- Unter Projekte "Create Model" starten
- Colab mit HUB linken

In [ ]:
# Training via Ultralytics HUB
%pip install -U ultralytics
from ultralytics import YOLO, checks, hub
checks()

Ultralytics YOLOv8.0.232 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.4/78.2 GB disk)


In [ ]:
# Training via Ultralytics HUB
hub.login('867d2f58d7c18fd4431f9edffa7e7995d1c5d23282')

model = YOLO('https://hub.ultralytics.com/models/pEKKIrLtlfOZmXjLk2gc')
results = model.train()

# **5. Detektion und Test**
Es gibt mehrere Möglichkeiten das Modell zu testen:
- Lokal mithilfe von `detect.py`
- Lokal mithilfe des Roboflow Packages
- Auf der [Roboflow](https://roboflow.com/?ref=ultralytics) Webseite
- Auf der [UltralyticsHub](https://hub.ultralytics.com/) Webseite

In [ ]:
# Entferne bisherige Testergebnisse
!rm -r runs/detect/i

In [ ]:
# Lokal mithilfe von `detect.py`: Vortrainiert mit ms coco
!python detect.py --weights yolov5s.pt --img 640 --conf 0.2 --source data/images/smartphone_test.jpg --name smartphone_test

In [ ]:
# Lokal mithilfe von `detect.py`: Smartphone
!python detect.py --weights trainedModels/3479_w_aug/weights/best.pt --img 640 --conf 0.5 --iou 0.35 --source data/images/smartphone_test.jpg --name smartphone_test

In [ ]:
# Lokal mithilfe von `detect.py`: bottle paper carton
!python detect.py --weights bottle_paper_carton_merge-3/result/run_1/weights/best.pt --img 640 --conf 0.4 --source data/images/b_p_c_test_1.jpg --name i/b_p_c_test_1
!python detect.py --weights bottle_paper_carton_merge-3/result/run_1/weights/best.pt --img 640 --conf 0.4 --source data/images/b_p_c_test_2.jpg --name i/b_p_c_test_2
!python detect.py --weights bottle_paper_carton_merge-3/result/run_1/weights/best.pt --img 640 --conf 0.4 --source data/images/b_p_c_test_3.jpg --name i/b_p_c_test_3

In [ ]:
# Lokal mithilfe von `detect.py`: Laptop
# !python detect.py --weights trainedModels/laptop_raw_run_2_colab/weights/best.pt --img 640 --conf 0.4 --iou 0.35 --source data/images/laptop.jpg --name laptop
!python detect.py --weights trainedModels/laptop_raw_run_1_ultralytics/weights/best.pt --img 640 --conf 0.4 --iou 0.35 --source data/images/laptop.jpg --name laptop

In [ ]:
# Lokal mithilfe des Roboflow Packages
from roboflow import Roboflow
rf = Roboflow(api_key="pEfHzLPIxO9w5xrhd4Zv")
project = rf.workspace().project("small_smartphone_dataset")
model = project.version(1).model

# !pwd
# !ls -la data/images/smartphone_test.jpg

# infer on a local image
# print(model.predict("/content/yolov5/data/images/smartphone_test.jpg", confidence=40, overlap=30).json())

# visualize your prediction
!cd small_smartphone_dataset-1/valid/images
model.predict("Datacluster-Labs-Phone-Dataset-17-_jpg.rf.0bafa13817017a23f922a556a103e8c5.jpg", confidence=30, overlap=30).save("prediction.jpg")

# **6. Export**
- Basis Befehl / Python Script = `export.py`
- Mehrere Argumente sind möglich:

```
# Auszug
--data
--weights
--imgsz
--batch-size
--device
[...]
--inplace
--keras
--optimize
```

- Default Werte sind gesetzt
- Beispielhafter Export Befehl: `!python export.py --weights ../datasets/train_journey/run/7/weights/best.pt --include tflite --imgsz 640 640 --data ../datasets/train_journey/../data.yaml`
- Visualisierung mithilfe von [Netron](https://netron.app/)

In [ ]:
# Export Beispiel
!python export.py --weights ../yolov5/trainedModels/b_p_c_m_w_aug_640_run_1/weights/best.pt --include tflite --imgsz 640 640

# **7. Weitere Links**
- [Ultralytics Hub Workshop](https://hub.ultralytics.com/projects/PFGKwK1dCBJPguNS72k7)